# Asset Classification (Supervised)

## Authors
- Bhargav Suryadevara (NVIDIA)
- Gorkem Batmaz (NVIDIA)

## Table of Contents 
* Introduction
* Dataset
* Reading in the datasets
* Training and inference
* References

# Introduction

In this notebook, we will show how to predict the function of a server with Windows Event Logs using cudf, cuml and pytorch. The machines are labeled as DC, SQL, WEB, DHCP, MAIL and SAP. The dependent variable will be the type of the machine. The features are selected from Windows Event Logs which is in a tabular format. This is a first step to learn the behaviours of certain types of machines in data-centres by classifying them probabilistically. It could help to detect unusual behaviour in a data-centre. For example, some compromised computers might be acting as web/database servers but with their original tag. 

This work could be expanded by using different log types or different events from the machines as features to improve accuracy. Various labels can be selected to cover different types of machines or data-centres.

## Library imports

In [1]:
import cudf
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as torch_optim
from torch.utils.dlpack import from_dlpack
from cuml.preprocessing import LabelEncoder
from cuml.preprocessing import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

10000 is chosen as the batch size to optimise the performance for this dataset. It can be changed depending on the data loading mechanism or the setup used. 

EPOCH should also be adjusted depending on convergence for a specific dataset. 

label_col indicates the total number of features used plus the dependent variable. Feature names are listed below.

In [2]:
batch_size = 10000
label_col = '19'
epochs = 15

#### Read the dataset into a GPU dataframe with `cudf.read_csv()` 

In [3]:
win_events_on_gpu = cudf.read_csv('win_events_18_features.csv')

The raw data had many other fields. Many of them were either static or mostly blank. After filtering those, there were 18 meaningful columns left that are listed below.

In [4]:
features = {
    "1" : "eventcode",
    "2" : "keywords",
    "3" : "privileges",
    "4" : "message",
    "5" : "sourcename", 
    "6" : "taskcategory",
    "7" : "account_for_which_logon_failed_account_domain",
    "8" : "detailed_authentication_information_authentication_package",
    "9" : "detailed_authentication_information_key_length",
    "10" : "detailed_authentication_information_logon_process",
    "11" : "detailed_authentication_information_package_name_ntlm_only",
    "12" : "logon_type",
    "13" : "network_information_workstation_name",
    "14" : "new_logon_security_id",
    "15" : "impersonation_level",
    "16" : "network_information_protocol",
    "17" : "network_information_direction",
    "18" : "filter_information_layer_name"
}

#### Categorize the columns
Categorical columns will be converted to numerical.

In [5]:
for col in win_events_on_gpu.columns:
    win_events_on_gpu[col] = win_events_on_gpu[col].astype('str')
    win_events_on_gpu[col] = win_events_on_gpu[col].fillna("NA")
    win_events_on_gpu[col] = LabelEncoder().fit_transform(win_events_on_gpu[col])

In [6]:
for col in win_events_on_gpu.columns:
    win_events_on_gpu[col] = win_events_on_gpu[col].astype('int16')

### Split the dataset into training and validation sets using cuML `train_test_split` function
Column 19 contains the ground truth about each machine's function that the logs come from. i.e. DC, SQL, WEB, DHCP, MAIL and SAP. Hence it will be used as a label.

In [7]:
X, val_X, Y, val_Y = train_test_split(win_events_on_gpu, label_col, train_size=0.9)

In [8]:
val_X.index = val_Y.index

In [9]:
val_X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
13136,1,0,0,14,0,4,25,5,0,0,0,3,1176,25,3,6,1,1
24921,0,1,0,15,0,4,22,0,0,5,0,6,0,4289,0,6,1,1
70496,0,1,0,15,0,4,22,0,0,5,0,1,932,2108,2,6,1,1
74882,0,1,0,15,0,4,22,4,1,7,2,6,1045,36,2,6,1,1
61138,0,1,0,15,0,4,22,0,0,5,0,1,0,4373,2,6,1,1


In [10]:
val_Y.head()

13136    4
24921    0
70496    2
74882    0
61138    0
Name: 19, dtype: int16

In [11]:
X, test_X, Y, test_Y = train_test_split(X,Y, train_size=0.9)

In [12]:
X.index = Y.index
test_X.index = test_Y.index

In [13]:
X.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
33693,0,1,0,15,0,4,22,0,0,5,0,1,932,1328,2,6,1,1
123,0,1,0,15,0,4,22,4,1,7,3,1,932,7980,3,6,1,1
28422,21,1,0,23,0,1,22,3,2,6,1,6,932,25,3,6,1,1
50761,0,1,0,15,0,4,22,4,1,7,2,1,591,34,3,6,1,1
33602,0,1,0,15,0,4,22,0,0,5,0,1,0,3929,2,6,1,1


In [14]:
Y.unique()

0    0
1    1
2    2
3    3
4    4
5    5
Name: 19, dtype: int16

### Print Labels
Making sure the test set contains all labels

In [15]:
test_Y.unique()

0    0
1    1
2    2
3    3
4    4
5    5
Name: 19, dtype: int16

### Embedding columns, check values in the columns
List the unique value counts in all the feature columns.

In [16]:
embedded_cols = {}
for col in X.columns:
    catergories_cnt = X[col].max()+2
    if catergories_cnt > 1:
        embedded_cols[col] = catergories_cnt
embedded_cols

{'1': 24,
 '2': 4,
 '3': 9,
 '4': 26,
 '5': 3,
 '6': 9,
 '7': 37,
 '8': 8,
 '9': 4,
 '10': 11,
 '11': 5,
 '12': 8,
 '13': 1408,
 '14': 8990,
 '15': 5,
 '16': 8,
 '17': 4,
 '18': 4}

In [17]:
X[label_col] = Y
val_X[label_col] = val_Y
test_X[label_col] = test_Y

#### Embedding
Feature columns will be embedded so that they can be used as categorical values. The limit can be changed depending on the accuracy of the dataset.

In [18]:
embedded_col_names = embedded_cols.keys()
embedded_col_names 

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18'])

In [19]:
embedded_cols.items()

dict_items([('1', 24), ('2', 4), ('3', 9), ('4', 26), ('5', 3), ('6', 9), ('7', 37), ('8', 8), ('9', 4), ('10', 11), ('11', 5), ('12', 8), ('13', 1408), ('14', 8990), ('15', 5), ('16', 8), ('17', 4), ('18', 4)])

In [20]:
embedding_sizes = [(n_categories, min(100, (n_categories+1)//2)) for _,n_categories in embedded_cols.items()]
embedding_sizes

[(24, 12),
 (4, 2),
 (9, 5),
 (26, 13),
 (3, 2),
 (9, 5),
 (37, 19),
 (8, 4),
 (4, 2),
 (11, 6),
 (5, 3),
 (8, 4),
 (1408, 100),
 (8990, 100),
 (5, 3),
 (8, 4),
 (4, 2),
 (4, 2)]

### Partition the dataframe
We have found that the way we partition the dataframes with a 10000 batch size gives us the optimum data loading capability. It can be adjusted for different sizes of datasets. 

In [21]:
def get_partitioned_dfs(df, batch_size):
    dataset_len = df.shape[0]
    prev_chunk_offset = 0
    partitioned_dfs = []
    while prev_chunk_offset < dataset_len:
        curr_chunk_offset = prev_chunk_offset + batch_size
        chunk = df.iloc[prev_chunk_offset:curr_chunk_offset:1]
        partitioned_dfs.append(chunk)
        prev_chunk_offset = curr_chunk_offset
    return partitioned_dfs

In [22]:
train_part_dfs = get_partitioned_dfs(X, batch_size)
val_part_dfs = get_partitioned_dfs(val_X, batch_size)
test_part_dfs = get_partitioned_dfs(test_X, batch_size)

In [23]:
train_part_dfs[0].head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
33693,0,1,0,15,0,4,22,0,0,5,0,1,932,1328,2,6,1,1,2
123,0,1,0,15,0,4,22,4,1,7,3,1,932,7980,3,6,1,1,1
28422,21,1,0,23,0,1,22,3,2,6,1,6,932,25,3,6,1,1,0
50761,0,1,0,15,0,4,22,4,1,7,2,1,591,34,3,6,1,1,0
33602,0,1,0,15,0,4,22,0,0,5,0,1,0,3929,2,6,1,1,0


In [24]:
del win_events_on_gpu
del X
del val_X
del test_X

## Check GPU availability

If there's a GPU, data is moved on to the GPU.

In [25]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [26]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [27]:
device = get_default_device()

In [28]:
device

device(type='cuda')

In [29]:
def bn_drop_lin(n_in, n_out, bn, p, actn):
    "Sequence of batchnorm (if `bn`), dropout (with `p`) and linear (`n_in`,`n_out`) layers followed by `actn`."
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

The bn_drop_lin function returns a sequence of batch normalization, dropout and a linear layer. This custom layer is usually used at the end of a model.

n_in represents the size of the input, n_out the size of the output, bn whether we want batch norm or not, p how much dropout, and actn (optional parameter) adds an activation function at the end.
Reference: https://github.com/fastai/fastai/blob/master/fastai/layers.py#L44

## Define Model Class

This class is the fast ai tabular model. More details can be found at https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6

In [30]:
class TabularModel(nn.Module):
    "Basic model for tabular data"
    
    def __init__(self, emb_szs, n_cont, out_sz, layers, drops, 
                 emb_drop, use_bn, is_reg, is_multi):
        super().__init__()
        
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(emb_drop)
        self.bn_cont = nn.BatchNorm1d(n_cont)
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont = n_emb,n_cont
        sizes = [n_emb + n_cont] + layers + [out_sz]
        actns = [nn.ReLU(inplace=True)] * (len(sizes)-2) + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+drops,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
                x = [e(x_cat[:,i]) for i,e in enumerate(model.embeds)]
                x = torch.cat(x, 1)
                x = self.emb_drop(x)
        if self.n_cont != 0:
            x_cont = self.bn_cont(x_cont) 
            x = torch.cat([x, x_cont], 1) if self.n_emb != 0 else x_cont
        x = self.layers(x)
        return x.squeeze()

In [31]:
model = TabularModel(embedding_sizes, 0, 6, [200,100], [0.001,0.01], emb_drop=0.04, is_reg=False,is_multi=True, use_bn=True)
to_device(model, device)

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(24, 12)
    (1): Embedding(4, 2)
    (2): Embedding(9, 5)
    (3): Embedding(26, 13)
    (4): Embedding(3, 2)
    (5): Embedding(9, 5)
    (6): Embedding(37, 19)
    (7): Embedding(8, 4)
    (8): Embedding(4, 2)
    (9): Embedding(11, 6)
    (10): Embedding(5, 3)
    (11): Embedding(8, 4)
    (12): Embedding(1408, 100)
    (13): Embedding(8990, 100)
    (14): Embedding(5, 3)
    (15): Embedding(8, 4)
    (16): Embedding(4, 2)
    (17): Embedding(4, 2)
  )
  (emb_drop): Dropout(p=0.04, inplace=False)
  (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=288, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.001, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): ReLU(inplace=True)
    (6): Ba

#### Define the Optimizer

Adam is the optimizer used in the training process; it is popular because it produces good results in various tasks. 
In its paper, computing the first and the second moment estimates and updating the parameters are summarized as follows

$$\alpha_{t}=\alpha \cdot \sqrt{1-\beta_{2}^{t}} /\left(1-\beta_{1}^{t}\right)$$

$$\theta_{t} \leftarrow \theta_{t-1}-\alpha_{t} \cdot m_{t} /(\sqrt{v_{t}}+\hat{\epsilon})$$

More detailson Adam can be found at https://arxiv.org/pdf/1412.6980.pdf

In [32]:
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

#### Training, evaluation and test functions
`train_model` function uses the dataframes to 
`val_loss` can be used with a validation function
`predict` function will be used with a test set

In [33]:
def train_model(model, optim, dfs):
    model.train()
    total = 0
    sum_loss = 0
    
    xb_cont_tensor = torch.zeros(0, 0)
    xb_cont_tensor.cuda()
    
    for df in dfs:
        batch = df.shape[0] 
        train_set = df.drop(label_col).to_dlpack()
        train_set = from_dlpack(train_set).long()
        
        output = model(train_set, xb_cont_tensor)
        train_label = df[label_col].to_dlpack()
        train_label = from_dlpack(train_label).long()
        
        loss = F.cross_entropy(output, train_label)   
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
        
    return sum_loss/total

#### Evaluation function

In [34]:
def val_loss(model, dfs):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    
    xb_cont_tensor = torch.zeros(0, 0)
    xb_cont_tensor.cuda()
    
    for df in dfs:
        current_batch_size = df.shape[0]
        
        val = df.drop(label_col).to_dlpack()
        val = from_dlpack(val).long()
        
        out = model(val, xb_cont_tensor)
        
        val_label = df[label_col].to_dlpack()
        val_label = from_dlpack(val_label).long()
        
        loss = F.cross_entropy(out, val_label)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        
        pred = torch.max(out, 1)[1]
        correct += (pred == val_label).float().sum().item()
    print("valid loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
    
    return sum_loss/total, correct/total

#### Test function

In [35]:
def predict(model, test_set):
    xb_cont_tensor = torch.zeros(0, 0)
    xb_cont_tensor.cuda()
    
    current_batch_size = test_set.shape[0]

    test_set = test_set.to_dlpack()
    test_set = from_dlpack(test_set).long()
    
    out = model(test_set, xb_cont_tensor)
    pred = torch.max(out, 1)[1].view(-1).tolist()
    return pred

In [36]:
def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs): 
        loss = train_model(model, optim, train_part_dfs)
        print("training loss: ", loss)
        val_loss(model, val_part_dfs)

In [37]:
def cleanup_cache():
    # release memory.
    torch.cuda.empty_cache()

## Training 

In [38]:
train_loop(model, epochs=epochs, lr=0.03, wd=0.00001)

/conda/envs/clx/lib/python3.7/site-packages/cudf/io/dlpack.py:74: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return libdlpack.to_dlpack(gdf_cols)


training loss:  0.920477888067111
valid loss 3.987 and accuracy 0.482
training loss:  0.40742612743359397
valid loss 0.962 and accuracy 0.736
training loss:  0.3070381289730306
valid loss 0.313 and accuracy 0.904
training loss:  0.2548799248944872
valid loss 0.257 and accuracy 0.913
training loss:  0.2262556504331736
valid loss 0.239 and accuracy 0.923
training loss:  0.20728669966486865
valid loss 0.219 and accuracy 0.926
training loss:  0.19436954608826432
valid loss 0.211 and accuracy 0.928
training loss:  0.1863015024571765
valid loss 0.208 and accuracy 0.928
training loss:  0.18150964236486616
valid loss 0.207 and accuracy 0.928
training loss:  0.1790416669681803
valid loss 0.203 and accuracy 0.930
training loss:  0.17679853713509194
valid loss 0.201 and accuracy 0.930
training loss:  0.17636516901632823
valid loss 0.209 and accuracy 0.930
training loss:  0.17703226489830243
valid loss 0.205 and accuracy 0.929
training loss:  0.17487056169265197
valid loss 0.215 and accuracy 0.930

In [39]:
def inference(model, test_part_dfs):
    pred_results = []
    true_results = []
    for df in test_part_dfs:
        pred_results.append(predict(model, df))
        true_results.append(df[label_col].values_host)                           
    pred_results = np.concatenate(pred_results).astype(np.int32)
    true_results = np.concatenate(true_results)
    f1_score_ = f1_score(pred_results, true_results,average='micro')
    print('micro F1 score: %s'%(f1_score_))
    return true_results, pred_results

In [40]:
true_results, pred_results = inference(model, test_part_dfs)
cleanup_cache()

micro F1 score: 0.9327287493232268


In [41]:
labels = ["DC","DHCP","MAIL","SAP","SQL","WEB"]
a = confusion_matrix(true_results, pred_results)

In [42]:
pd.DataFrame(a, index=labels, columns=labels)

,DC,DHCP,MAIL,SAP,SQL,WEB
DC,3115,28,14,1,64,4
DHCP,62,604,1,0,18,1
MAIL,5,0,2394,4,6,0
SAP,16,0,2,118,21,0
SQL,168,0,5,11,617,15
WEB,22,0,0,0,29,43


The confusion matrix shows that some machines' function can be predicted really well, whereas some of them need more tuning or more features. This work can be improved and expanded to cover individual data-centres to create a realistic map of the network using ML by not just relying on the naming conventions. It could also help to detect more prominent scale anomalies like multiple machines, not acting per their tag.

## References:
* https://github.com/fastai/fastai/blob/master/fastai/tabular/models.py#L6
* https://jovian.ml/aakashns/04-feedforward-nn
* https://www.kaggle.com/dienhoa/reverse-tabular-module-of-fast-ai-v1
* https://github.com/fastai/fastai/blob/master/fastai/layers.py#L44